In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','大庆市'))
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [6]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [8]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

# 筛选得出采样影像

In [9]:
image_area = landsat_images.mosaic().clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})
# print(areas.getInfo())

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',ee.Number(areas.get('B2')).divide(2)))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

11
['2018-04-28', '2018-05-30', '2018-07-01', '2018-08-02', '2018-08-18', '2018-09-19', '2018-10-05', '2018-10-21', '2018-11-06', '2018-11-22', '2018-12-08']


# 样本采集函数的定义

In [ ]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# # Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
# Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
# Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
# Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# # 三者的交集
# permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
# landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
# seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
# 两者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    permanent = permanentWaterExtent.clip(image.geometry()).remap([0,1],[0,2]).rename('waterclass')
    land = landExtent.clip(image.geometry())
    awei = image.clip(roi).updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': roi, 
        'scale': 30, 
        'bestEffort': True
    })
    season_class = awei.gt(ee.Image.constant(intervalMean.get('AWEI'))).rename('waterclass')
    seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image.geometry()) 
    percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
    all_points = percentile_waterclass.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 2000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points

In [ ]:
# 总的样本点数
points_collection = sample_image.map(imageSample).flatten()
print('total sample number:{}'.format(points_collection.size().getInfo()))
print(points_collection.first().getInfo())

# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# # 非水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

In [ ]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

image9 = water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_119028_20180428'))
Map.addLayer(image9.clip(roi),visParams,'image9')

In [ ]:
permanent = permanentWaterExtent.clip(image9.geometry()).remap([0,1],[0,2]).rename('waterclass')
land = landExtent.clip(image9.geometry())
season_class = water_index(maskL8sr(image9.clip(roi))).updateMask(seasonWaterExtent).select('AWEI').gt(0).rename('waterclass')
seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image9.geometry()) 
percentile_waterclass_0 = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
Map.addLayer(percentile_waterclass_0.selfMask(),{'palette':['blue']},'percentile waterclass_0')

In [ ]:
image_9 = image9.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
Map.addLayer(image_9.selfMask(),{'palette':['green']}, 'focal_mode')

In [ ]:
permanent = permanentWaterExtent.clip(image9.geometry()).remap([0,1],[0,2]).rename('waterclass')
land = landExtent.clip(image9.geometry())
season_class = water_index(maskL8sr(image9.clip(roi))).updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(-674.8320019543804)).rename('waterclass')

seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass').clip(image9.geometry()) 
percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
Map.addLayer(percentile_waterclass.selfMask(),{'palette':['red']},'percentile waterclass')